In [1]:
%pip install sentence-transformers qdrant-client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 913.8 kB/s eta 0:00:00 0:00:01
  Using cached safetensors-0.4.5-cp311-cp311-macosx_11_0_arm64.whl.metadata (3.8 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 3.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.2/267.2 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 43.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 35.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 55.8 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.8/417.8 kB 16.2 MB/s eta 0:00:00
Using cached safetensors-0.4.5-cp311-cp311-macosx_11_0_arm64.whl (381 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 47.0 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4

In [2]:
from sentence_transformers import SentenceTransformer

# Загружаем модель
model = SentenceTransformer('DeepPavlov/rubert-base-cased')

No sentence-transformers model found with name DeepPavlov/rubert-base-cased. Creating a new one with mean pooling.


config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [16]:
from tqdm import tqdm
import pandas as pd

In [19]:
df_articles = pd.read_csv('../../data/df_articles.csv').dropna()
df_articles['content'] = df_articles['title'] + df_articles['content']

df_articles_analytics = pd.read_csv('../../data/df_articles_analytics.csv').dropna()
df_articles_analytics['content'] = df_articles_analytics['title'] + df_articles_analytics['content']

data = df_articles['content'].to_list() + df_articles_analytics['content'].to_list()
with open('../../data/courses.txt', 'r') as f:
    data += list(f.read().split('\n\n\n\n'))

In [20]:
len(data)

525

In [21]:
def create_embeddings(data, model):
    embeddings = []
    for item in tqdm(data):
        embedding = model.encode(item, convert_to_numpy=True)
        embeddings.append(embedding)
    return embeddings

In [22]:
embeddings = create_embeddings(data, model)
print("Эмбеддинги созданы.")

100%|██████████| 525/525 [00:28<00:00, 18.38it/s]

Эмбеддинги созданы.


In [24]:
from qdrant_client import QdrantClient
from qdrant_client.models import PointStruct

# Подключение к Qdrant
qdrant_client = QdrantClient("http://localhost:6333")  # Убедитесь, что Qdrant запущен

# Создание коллекции
qdrant_client.recreate_collection(
    collection_name="financial_assistant",
    vector_size=embeddings[0].shape[0],  # Размерность вектора
    distance="Cosine"
)


TypeError: QdrantClient.recreate_collection() missing 1 required positional argument: 'vectors_config'

In [ ]:

# Загрузка данных в Qdrant
points = [
    PointStruct(
        vector=item["embedding"].tolist()
    )
    for item in embeddings
]

qdrant_client.upsert(collection_name="financial_assistant", points=points)
print("Данные загружены в Qdrant.")